<a href="https://colab.research.google.com/github/taka-too/home_num_pdf_gen/blob/main/home_num_pdf_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Homeナンバー自動入力ツールの使い方**

### **📌 0. このツールについて**
- 父母会配布物に記載するHomeナンバーを自動で付番し、タイトルをつけます。
- Colabを実行すると操作はGUIでできますので、直感的に操作できます。
- エクセルにHomeナンバーを記載し読み込ませ実行するとpdfが生成されます。
- 差し込み印刷ができるプリンターで配布物にこのpdfを印刷してください。

---

### **📌 1. 実行環境の準備**
- Google Colab でこのノートブックを開きます。
- コードをすべて実行（`Shift + Enter` でセルを順に実行）
- 起動時、必要なライブラリを仮想環境にダウンロードしますので、少し時間がかかります。

---

### **📌 2. 設定項目の入力**
実行すると現れる下記のGUIにて以下の項目を設定します：

| 設定項目 | 説明 | 初期値 |
|----------|------|------|
| **タイトル** | PDF の上部に表示されるタイトル | `これは父母会の配布物です` |
| **フォントサイズ** | テキストのサイズ（8〜24） | `12` |
| **X 位置** | タイトルの X 座標（50〜200） | `130` |
| **Y 位置** | タイトルの Y 座標（0〜50） | `10` |
| **出力ファイル名** | ダウンロード時の PDF ファイル名 | `generated_document.pdf` |

![(https://github.com/user-attachments/assets/668193cf-ceb8-4c18-b2d7-077c914aae26)](https://github.com/user-attachments/assets/668193cf-ceb8-4c18-b2d7-077c914aae26)

---

### **📌 3. Excel ファイルのアップロード**
- **「Excelをアップロード」ボタン** をクリックし、をクリックし、入力する番号を並べた Excelファイル (.xlsx) を選択。

[エクセルファイルの例](https://github.com/taka-too/home_num_pdf_gen/raw/refs/heads/main/example_home_num.xlsx)

- **アップロードが完了すると、ファイル名が表示されます**。

---

### **📌 4. PDF 作成＆ダウンロード**
1. **「PDFを作成」ボタン** をクリック
2. **ユーザーが指定したファイル名でダウンロードが自動開始**（Google Drive には保存されません）
3. **ダウンロードしたpdfを差し込み印刷ができるプリンターで印刷してください**

---

### **📌 5. Note **
- **Google Drive にファイルを保存しません**（すべてメモリ上で処理）。
- **ダウンロード後、Google Colab にはデータが一切残りません**(Colab のセッションが終了すると一時ファイルも削除)

---



In [ ]:
# 必要なライブラリをインストール
!pip install fpdf2

import pandas as pd
from fpdf import FPDF
import io
import tempfile
import os
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

# Google Colab で実行するためのフォントダウンロード
!apt-get install fonts-noto-cjk -y

# フォントのパスを設定
font_path = "/usr/share/fonts/opentype/noto/NotoSerifCJK-Regular.ttc"

# ユーザー入力フィールドを作成
title_input = widgets.Text(value="これは父母会の配布物です", description="タイトル:")
font_size_input = widgets.IntSlider(value=12, min=8, max=24, step=1, description="フォントサイズ:")
x_position_input = widgets.IntSlider(value=130, min=50, max=200, step=10, description="X位置:")
y_position_input = widgets.IntSlider(value=10, min=0, max=50, step=1, description="Y位置:")
output_filename_input = widgets.Text(value="generated_document.pdf", description="出力ファイル名:")

# ファイルアップロードボタンを作成
upload_button = widgets.FileUpload(accept=".xlsx", multiple=False, description="Excelをアップロード")

# アップロードされたファイル名を表示するテキストボックス
uploaded_file_label = widgets.Text(value="", description="ファイル名:", disabled=True)

# 実行ボタンを作成
run_button = widgets.Button(description="PDFを作成")

# ユーザーインターフェースを表示
display(title_input, font_size_input, x_position_input, y_position_input, output_filename_input, upload_button, uploaded_file_label, run_button)

# ファイルが選択されたときに即時ファイル名を表示する関数
def on_file_upload(change):
    if upload_button.value:
        uploaded_filename = list(upload_button.value.keys())[0]
        uploaded_file_label.value = uploaded_filename

# ファイルアップロード時にファイル名を表示
upload_button.observe(on_file_upload, names="value")

def create_pdf(input_excel_stream, output_filename, title, font_size, x_pos, y_pos):
    # Excelファイルをメモリ上で読み込む
    df = pd.read_excel(input_excel_stream, usecols=[0], header=None, names=["家庭番号"])

    # PDF生成開始（メモリ上）
    pdf = FPDF()
    pdf.set_margins(0, 0, 0)  # 上下左右の余白をゼロに設定
    pdf.set_auto_page_break(auto=True, margin=0)  # 自動改ページ時の余白もゼロに設定

    # フォントの登録
    pdf.add_font("NotoSerifCJK", fname=font_path)

    for index in range(len(df)):
        family_number = df.iloc[index]["家庭番号"]

        # タイトルと家庭番号を結合
        title_and_number = f"{title} {family_number}"

        # 新しいページを追加
        pdf.add_page()
        pdf.set_font("NotoSerifCJK", size=font_size)  # ユーザー指定のフォントサイズ

        # 右上に結合テキストを配置
        pdf.set_xy(x_pos, y_pos)  # ユーザー指定の位置
        pdf.cell(70, 10, title_and_number, align="R")  # 右揃えで表示

    # `/tmp/` にユーザー指定のファイル名で保存
    temp_pdf_path = os.path.join(tempfile.gettempdir(), output_filename)

    pdf.output(temp_pdf_path)  # ファイルを /tmp/ に保存

    # PDFをダウンロード
    files.download(temp_pdf_path)

def on_run_button_clicked(b):
    if upload_button.value:
        uploaded_filename = list(upload_button.value.keys())[0]
        uploaded_file_label.value = uploaded_filename  # ファイル名を表示

        # メモリ上でExcelデータを扱う
        input_excel_stream = io.BytesIO(upload_button.value[uploaded_filename]["content"])

        # PDFを作成してダウンロード
        create_pdf(input_excel_stream, output_filename_input.value, title_input.value, font_size_input.value, x_position_input.value, y_position_input.value)

# 実行ボタンが押されたときの処理を設定
run_button.on_click(on_run_button_clicked)
